In [6]:
import numpy as np
import pandas as pd
import requests # to make TMDB API calls
import locale # to format currency as USD
locale.setlocale( locale.LC_ALL, '' )
api_key = '07c743da178e65cad9e43897e89fc202'
import ast
import json

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [38]:
dict_columns = ['belongs_to_collection', 'genres', 'production_companies',
                'production_countries', 'spoken_languages', 'Keywords', 'cast', 'crew']

def text_to_dict(df):
    for column in dict_columns:
        df[column] = df[column].apply(lambda x: {} if pd.isna(x) else ast.literal_eval(x) )
    return df

In [22]:
train_final = pd.DataFrame()
train_final['iscollect'] = train['belongs_to_collection'].notnull().astype('int')
train_final['revenue'] = np.log(train['revenue'])

test_final = pd.DataFrame()
test_final['iscollect'] = test['belongs_to_collection'].notnull().astype('int')

In [23]:
genres = train['genres']
unique_genres = set()
for gen in genres:
    try:
        names = json.loads(gen.replace("'",'"'))
        name_list = [name['name'] for name in names]
        unique_genres.update(name_list)
    except:
        pass
    
train_final = train_final.reindex(list(unique_genres)+['iscollect','revenue'],fill_value=0,axis='columns')

for i, gen in enumerate(genres):
    try:
        names = json.loads(gen.replace("'",'"'))
        name_list = [name['name'] for name in names]
        for n in name_list:
            train_final.at[i,n] = 1
    except:
        pass

    
test_final = test_final.reindex(list(unique_genres)+['iscollect'],fill_value=0,axis='columns')

for i, gen in enumerate(genres):
    try:
        names = json.loads(gen.replace("'",'"'))
        name_list = [name['name'] for name in names]
        for n in name_list:
            use_data.at[i,n] = 1
    except:
        pass

In [24]:
sig_diff = []
for g in tr_unique_genres:
    gen_rev = train_final.loc[train_final[g]==1,'revenue']
    nongen_rev = train_final.loc[train_final[g]==0,'revenue']
    diff = np.mean(gen_rev)-np.mean(nongen_rev)
    if abs(diff) < 0.4 or len(gen_rev) < 30:
        sig_diff.append(g)

In [25]:
train_final.drop(columns=sig_diff,inplace=True)
test_final.drop(columns=sig_diff,inplace=True)

In [36]:
train_final['popularity'] = train['popularity']
test_final['popularity'] = test['popularity']

In [39]:
train = text_to_dict(train)
test = text_to_dict(test)

In [ ]:
def getRev(row):
    response = requests.get('https://api.themoviedb.org/3/find/'+ row.imdb_id + '?api_key=' + api_key +'&language=en-US&external_source=imdb_id')
    if 'movie_results' in response.json().keys():
        if len(response.json()['movie_results']) >= 1:
            tmdbid = response.json()['movie_results'][0]['id']
        else:
            return 0
    else:
        return 0
    response = requests.get('https://api.themoviedb.org/3/movie/' + str(tmdbid) + '?api_key=' +  api_key + '&language=en-US')
    if 'revenue' in response.json().keys():
        return response.json()['revenue']
    else: 
        return 0

revenues = test.apply(lambda row: getRev(row), axis = 1)
test_final['revenue'] = revenues

In [ ]:
train_final['cast'] = train.apply(lambda row: ast.literal_eval(row.cast) if type(row.cast) == str else 0, axis = 1)
train_final['crew'] = train.apply(lambda row: ast.literal_eval(row.crew) if type(row.crew) == str else 0, axis = 1)
train_final['production_companies'] = train.apply(lambda row: ast.literal_eval(row.production_companies) if type(row.production_companies) == str else 0, axis = 1)

test_final['cast'] = test.apply(lambda row: ast.literal_eval(row.cast) if type(row.cast) == str else 0, axis = 1)
test_final['crew'] = test.apply(lambda row: ast.literal_eval(row.crew) if type(row.crew) == str else 0, axis = 1)
test_final['production_companies'] = test.apply(lambda row: ast.literal_eval(row.production_companies) if type(row.production_companies) == str else 0, axis = 1)

In [ ]:
def actorReturn(row):
    if type(row.cast) == list:
        if len(row.cast) >= 2:
            actors = {
                "actor1": row.cast[0]['name'],
                "actor2": row.cast[1]['name']
            }
            return actors
        elif len(row.cast) == 1:
            actors = {
                "actor1": row.cast[0]['name'],
            }
            return actors
        else:
                    return 0
    else:
        return 0
    
def returnCrew(crew):
    crewDict = {
        'Director': None,
        'Producer': None
    }
    direc = 0
    prod = 0
    for x in crew:
        #print(crewDict)
        if x['job'] == 'Director':
            crewDict['Director'] = x['name']
            direc = 1
        elif x['job'] == 'Producer':
            crewDict['Producer'] = x['name']
            prod = 1
        if direc > 0 and prod > 0:
            return crewDict
    return crewDict

In [ ]:
train_final['rel_actors'] = train.apply(lambda row: actorReturn(row), axis =1) 
train_final['rel_crew'] = train.apply(lambda row: returnCrew(row.crew) if type(row.crew) == list else {},  axis =1)

test_final['rel_actors'] = t.apply(lambda row: actorReturn(row), axis =1) 
test_final['rel_crew'] = train.apply(lambda row: returnCrew(row.crew) if type(row.crew) == list else {},  axis =1)